In [2]:
###DATA LOADING MODULE###
from langchain_core.documents import Document

In [3]:
doc = Document(
    page_content = "This is the content of the document",
    metadata ={
        "author": "Parth Sharma",
        "created_on": "2025-09-21",
        "page": 1
    }
)
print(doc)

page_content='This is the content of the document' metadata={'author': 'Parth Sharma', 'created_on': '2025-09-21', 'page': 1}


In [3]:
## File/document loader
from langchain_community.document_loaders import TextLoader

loader = TextLoader("../data/textfiles/llm.txt")
document = loader.load()
meta = document[0].metadata
meta.update({"author":"Parth Sharma"})
document[0].metadata = meta
print(document[0].metadata)

{'source': '../data/textfiles/llm.txt', 'author': 'Parth Sharma'}


In [5]:
## Directory loader
from langchain_community.document_loaders import DirectoryLoader
loader = DirectoryLoader("../data/textfiles",glob ="*.txt",loader_cls=TextLoader)
documents = loader.load()
print(documents)

[Document(metadata={'source': '..\\data\\textfiles\\llm.txt'}, page_content='LLM means large language model. An LLM is an artificial intelligence system designed to understand and generate human language, typically trained on vast amounts of text data.'), Document(metadata={'source': '..\\data\\textfiles\\rag.txt'}, page_content='RAG stands for Retrieval-Augmented Generation. It is an AI technique that combines information retrieval with language generation, allowing models to fetch relevant documents and generate responses based on both retrieved data and their own knowledge.')]


In [6]:
## Directory loader
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_community.document_loaders import DirectoryLoader
loader = DirectoryLoader("../data/pdffiles",glob ="*.pdf",loader_cls=PyMuPDFLoader)
pdf_documents = loader.load()
print(pdf_documents)

c:\Users\ASUS\Desktop\RAG-Data_ingestion\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Document(metadata={'producer': 'pikepdf 8.15.1', 'creator': 'arXiv GenPDF (tex2pdf:)', 'creationdate': '', 'source': '..\\data\\pdffiles\\embeddings.pdf', 'file_path': '..\\data\\pdffiles\\embeddings.pdf', 'total_pages': 17, 'format': 'PDF 1.5', 'title': 'Harnessing the Universal Geometry of Embeddings', 'author': 'Rishi Jha; Collin Zhang; Vitaly Shmatikov; John X. Morris', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}, page_content='arXiv:2505.12540v3  [cs.LG]  25 Jun 2025\nHarnessing the Universal Geometry of Embeddings\nRishi Jha\nCollin Zhang\nVitaly Shmatikov\nJohn X. Morris\nDepartment of Computer Science\nCornell University\nAbstract\nWe introduce the first method for translating text embeddings from one vector\nspace to another without any paired data, encoders, or predefined sets of matches.\nOur unsupervised approach translates any embedding to and from a universal latent\nrepresentation (i.e., a universal semantic

##### TODO - Excel parsing and DB file parsing 

In [7]:
### TODO

Embeddings 

In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict,Tuple, Any
from sklearn.metrics.pairwise import cosine_similarity
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter

c:\Users\ASUS\Desktop\RAG-Data_ingestion\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
class EmbeddingModel:
    """ Class to handle text embeddings using SentenceTransformer.

        Args:
            model_name (str): Name of the pre-trained model to use. Defaults to 'all-MiniLM-L6-v2'.
    """
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        self.model_name = model_name
        self.model = None
        self._load_model() ## we write protected method to load the model. this method is not accessible outside the class.
    
    def _load_model(self):
        """Load the SentenceTransformer model."""
        try:
            self.model = SentenceTransformer(self.model_name)
            print(f"Loaded model: {self.model_name}")
            print(f"model_dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise e

    
    def parse_documents(self,path: str) -> List[Document]:
        """Parse documents from the given directory.

            Args:
                path (str): Path to the directory containing text files.
        """
        if not os.path.isdir(path):
            raise ValueError(f"Provided path {path} is not a valid directory.")
        
        loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
        documents = loader.load()
        return documents
    
    def batch_embed_texts(self, texts: List[str], batch_size: int = 32) -> np.ndarray:
        """Embed texts in batches to handle large datasets.

            Args:
                texts (List[str]): List of texts to be embedded.
                batch_size (int): Size of each batch. Defaults to 32.
        """
        if not self.model:
            raise ValueError("Model not loaded.")
        
        embeddings =[]
        for i in range(0,len(texts),batch_size):
            batch_texts = texts[i:i+batch_size]
            batch_embeddings = self.model.encode(batch_texts, convert_to_numpy=True)
            embeddings.append(batch_embeddings)
        return np.vstack(embeddings)
    
    def chunking(self, path:str, chunk_size: int = 500, overlap: int = 50) -> Tuple[List[Document], np.ndarray]:
        """Chunk the text into smaller pieces.

            Args:
                text (str): The text to be chunked.
                chunk_size (int): The size of each chunk. Defaults to 500.
                overlap (int): The number of overlapping characters between chunks. Defaults to 50.

            Returns:
                List[str]: List of text chunks.
        """
        All_texts = self.parse_documents(path) ## It will bydefault only split page content and not metadata.
        text_splitter = RecursiveCharacterTextSplitter( 
            chunk_size=chunk_size,
            chunk_overlap=overlap,
            separators=["\n\n", "\n", " ", ""]
        )
        chunks = text_splitter.split_documents(All_texts)
        embedding = self.batch_embed_texts([chunk.page_content for chunk in chunks])
        return chunks, embedding
        
    



In [17]:
embedding_model = EmbeddingModel()
chunks, embeddings_ = embedding_model.chunking("../data/pdfFiles")
print(f"Number of chunks: {len(chunks)}")
print(f"Embedding shape: {embeddings_.shape}")

Loaded model: all-MiniLM-L6-v2
model_dimension: 384
Number of chunks: 134
Embedding shape: (134, 384)


In [4]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        
        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2
Model loaded successfully. Embedding dimension: 384


Vector DB

In [5]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "pdfFiles", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore

: 

In [23]:

vectorstore.add_documents(chunks, embeddings_)     

: 